# Poisson problem: Resolution example with a delta de Dirac  as a source 

Recall that the Dirac distribution acts on test functions as: $delta_{x_0}(\phi) = \phi(x_0)$

In the setting where we are working this definition does not make sense because the test functions are in $H^1(\Omega)$ or $H^1_0(\Omega)$ or a mixture of them, but then, unless in 1-dimension, its elements are not defined at given points of the manifold. It is also the case that the solution, which in two dimensions goes like $\phi(r) = qln(r)$ is infinite at the origin, and so not represented as a function in $H^1$.

Nevertheless it is possible to get a good approximation to it. 


The problem to solve is:


\begin{align*}
-\Delta u &= f + q\delta(x_0) \;\;\;\;\; \text{in } \Omega \\
u &= h \;\;\;\;\; \text{in } \partial \Omega_{ext}, \\
\end{align*}

Where we should think of it as a relation among distributions. 

The weak version of it is:

Find $u$ en $H^1_0(\Omega)$ (so we are impossing homogeneous boundary conditions on the $\partial \Omega_{int}$) such that:

$$
\int_{\Omega} \nabla v \cdot \nabla u \; d\Omega 
- \int_{\Omega} v \; f \; d\Omega + q*v(x_0)
= 0 \;\;\;\;\; \forall v \;\; \in H^1_0(\Omega)
$$

As noted above this problem is not well defined for all $v \in $H^1_0(\Omega)$ for unless in 1-dimension their elements are not functions. 
But for the restriction to smoother functions it is well defined.

So solve this problem we shall use the infrastructure provided by the Julia library `Gridap.jl`. Julia provides a *"function"* for Dirac's delta.

In [ ]:
using Gridap
using GridapMakie, Makie, GLMakie #Para graficar 
GLMakie.activate!(inline=true) # For windows on the notebook itself. Comment out if you want the as pop-out plots.
using FileIO #Gráficos y salidas
using GridapGmsh
#mkdir("models") #if the directories are not yet created, create them!
#mkdir("images") #if the directories are not yet created, create them!
#import Gmsh = gmsh # NEVER use using Gmsh!

To solve the problem we are going to use a special grid where we have taken very small triangles near zero so that we can represent appropriately the approximation we are going to obtain. We also have choosen that there will be a node at the origin, that is where the delta has support. The way to build it can be found at "models/central_point.geo"

In [ ]:
model = GmshDiscreteModel("models/central_point.msh")

Out of the mesh we create the triangulation. 

In [ ]:
Ω = Triangulation(model)

In [ ]:
degree = 2
dΩ = Measure(Ω,degree)


Plot it:

In [ ]:
fig, ax = plot(Ω)
ax.aspect = AxisAspect(1)
wireframe!(Ω, color=:black, linewidth=1)
scatter!(Ω, marker=:star8, markersize=4, color=:blue)
fig

We also plot the boundaries to make sure they are correct: 

In [ ]:
#labels = get_face_labeling(model)
#add_tag_from_tags!(labels,"diri",[8])
#add_tag_from_tags!(labels,"point",[7]);

#boundary_tags = "diri"
boundary_tags = ["boundary"]

Γ = BoundaryTriangulation(model,tags=boundary_tags)
dΓ = Measure(Γ,degree)
fig, ax = plot(Γ, linewidth=8)
ax.aspect = AxisAspect(1)
wireframe!(Γ, color=:black, linewidth=1)
fig


Once the grid is defined with all the needed attributes we define the finite elements we shall be using. For this case we shall use *lagrangian* finite elements of **orden 2** that satisfy the homogeneous Dirichlet condition at the boundary $\partial \Omega_{boundary}$. 

In [ ]:
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)

dirichlet_tags= ["boundary"] 

V = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags = dirichlet_tags)
#V = TestFESpace(model,reffe;conformity=:L2,dirichlet_tags = dirichlet_tags) # for this weaker space the inversion does not work.


Now we define the trial space also with homogeneous Dirichlet conditions. 

In [ ]:
U = TrialFESpace(V,[0.0])


So now we are in conditions of defining the weak problem in its abstract form: 

In [ ]:
f(x) = 0 # for this case we take the continuous source to vanish, but we could also put a continuous charge distribution.  
#δ_point = DiracDelta{0}(model,tags=["mypoint1","mypoint2"])
δ = DiracDelta{0}(model,tags="point")

q = 1

a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ  # In a(u,v) goes all the u dependence (the unknown).  

b(v) = ∫(v*f )*dΩ + δ( q⋅v ) # In b(v) all the sources 


From this point on the package **Gridap.jl** generates a system of the form: $Ax=b$ and solves it for $u$ in the finite element version given. It first computes the matrices:

In [ ]:
op = AffineFEOperator(a,b,U,V)

Then it solves with some of the possible known strategies. Here we give two of them, the first is just an L-U decomposition (Lower triangular Unitary). The second it the `/` operator of Julia, which in general would use the most powerful method for the case.

In [ ]:
#ls = LUSolver()
ls = BackslashSolver()
solver = LinearFESolver(ls)

And we solve it. We call the solution $u_h$.

In [ ]:
uh = solve(solver,op)

Now we can plot the approximation found and save it to check it in another application. The saved files can be seen either with Paraview or VisIt.

In [ ]:
fig, ax, plt = plot(Ω, uh)
ax.aspect = AxisAspect(1)
Colorbar(fig[2,1], plt, vertical=false)
fig


In [ ]:
writevtk(Ω,"images/delta",cellfields=["ϕ" => uh])
writevtk(Ω,"images/E_delta",cellfields=["E" => -∇(uh)])
